# Retrieval Augmented Generation

In [ ]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

# Define MariTalk API key and LLM model
MARITALK_API_KEY = os.getenv('MARITALK_API_KEY')
MARITALK_LLM_MODEL = "sabia-3"

# Define Langsmith API key and tracing
LANGSMITH_API_KEY = os.getenv('LANGSMITH_API_KEY')
LANGSMITH_TRACING = os.getenv('LANGSMITH_TRACING')

# Define Pinecone API key
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

In [3]:
from langchain_community.chat_models import ChatMaritalk

# Initialize MariTalk LLM model
llm = ChatMaritalk(
    model=MARITALK_LLM_MODEL,
    api_key=MARITALK_API_KEY,
    max_tokens=1000,
)

In [ ]:
%pip install -qU langchain-ollama
%pip install -qU langchain-pinecone

In [5]:
from langchain_ollama import OllamaEmbeddings

# Define Ollama embeddings
embeddings = OllamaEmbeddings(model="llama3")

In [ ]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone

index_name = "ollama-llama3-capiara-algorithm-mentor"

pinecone = Pinecone(api_key=PINECONE_API_KEY)
index = pinecone.Index(index_name)

vector_store = PineconeVectorStore(embedding=embeddings, index=index)

In [ ]:
%pip install --quiet bs4

In [ ]:
%pip install langgraph


In [ ]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()